In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/data/

In [ ]:
!pip install tsfresh

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import random
import math
from tsfresh.feature_extraction import extract_features, ComprehensiveFCParameters
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.discriminant_analysis  import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [7]:

event_names = {0: 'Normal',
                1: 'Abrupt Increase of BSW',
                2: 'Spurious Closure of DHSV',
                3: 'Severe Slugging',
                4: 'Flow Instability',
                5: 'Rapid Productivity Loss',
                6: 'Quick Restriction in PCK',
                7: 'Scaling in PCK',
                8: 'Hydrate in Production Line'
               }

fc_parameters = {
    "mean": None,
    "median": None,
    "variance": None,
    "skewness": None,
    "kurtosis": None,
    "fft_aggregated": [{'aggtype': 'centroid'}, {'aggtype': 'variance'}, {'aggtype': 'skew'}, {'aggtype': 'kurtosis'}],
    "maximum": None,
    "minimum": None,
    "quantile": [{'q': 0.1}, {'q': 0.2}, {'q': 0.3}, {'q': 0.4}, {'q': 0.6}, {'q': 0.7}, {'q': 0.8}, {'q': 0.9}],
    "variation_coefficient": None,
    "mean_change": None,
    "mean_second_derivative_central": None,
    "friedrich_coefficients": [{'coeff': 1, 'm': 1, 'r': 30}, {'coeff': 1, 'm': 3, 'r': 30}],
}

scaler = preprocessing.StandardScaler()


Delete rows without a class

In [ ]:
for f in range(0, 9):
  path_dir = str(f)
  os.mkdir(path_dir + path_dir)
  for file in os.listdir(path_dir):
    df = pd.read_csv(path_dir + "/" + file)
    df = df[df['class'].notna()]
    df.to_csv(path_dir + path_dir + "/" + file, index=False)

### Downsampling

In [ ]:
for f in range(0, 9):
  str_f = str(f)
  path_dir = str_f + str_f
  os.mkdir(f"downsampled/{str_f}")
  for file in os.listdir(path_dir):
    df = pd.read_csv(path_dir + "/" + file)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.resample('10S', on='timestamp').mean()
    df.to_csv(f"downsampled/{str_f}/{file}", index=False)

### Time window division

In [ ]:
os.mkdir("divided300")

### hiperparameter
time_window_size = 30

for f in range(0, 9):
  if f == 7:
    continue
  path_dir = str(f)
  os.mkdir(f"divided300/{f}")
  for file in os.listdir(f"downsampled/{f}"):
    if file.startswith('DRAWN'):
      continue
    df = pd.read_csv(f"downsampled/{path_dir}/{file}")
    df.drop(columns=['P-JUS-CKGL', 'T-JUS-CKGL', 'QGL'], inplace=True)
    df['id'] = 0
    row_count = len(df.index)
    ts_count = 1
    i = 0
    while i < row_count:
      if i + time_window_size < row_count:
        if df['class'].iloc[i] != df['class'].iloc[i+time_window_size]:
          tmp = df.loc[i:(i+time_window_size)]
          maxi = tmp[['class']].value_counts().index[0]
          tmp.loc[df['class'] == maxi, 'id'] = ts_count
          df.loc[i:(i+time_window_size), :] = tmp[:]
        else:
          df.loc[i:(i+time_window_size), 'id'] = ts_count
      elif row_count - i >= time_window_size / 2:
        df.loc[i:(row_count-1), 'id'] = ts_count
      else:
        break
      i += time_window_size
      ts_count += 1

    df.to_csv(f"divided300/{path_dir}/{file}", index=False)


In [ ]:
os.mkdir("divided600")

### hiperparameter
time_window_size = 60

for f in range(0, 9):
  if f == 7:
    continue
  path_dir = str(f)
  os.mkdir(f"divided600/{f}")
  for file in os.listdir(f"downsampled/{f}"):
    if file.startswith('DRAWN'):
      continue
    df = pd.read_csv(f"downsampled/{path_dir}/{file}")
    df.drop(columns=['P-JUS-CKGL', 'T-JUS-CKGL', 'QGL'], inplace=True)
    df['id'] = 0
    row_count = len(df.index)
    ts_count = 1
    i = 0
    while i < row_count:
      if i + time_window_size < row_count:
        if df['class'].iloc[i] != df['class'].iloc[i+time_window_size]:
          tmp = df.loc[i:(i+time_window_size)]
          maxi = tmp[['class']].value_counts().index[0]
          tmp.loc[df['class'] == maxi, 'id'] = ts_count
          df.loc[i:(i+time_window_size), :] = tmp[:]
        else:
          df.loc[i:(i+time_window_size), 'id'] = ts_count
      elif row_count - i >= time_window_size / 2:
        df.loc[i:(row_count-1), 'id'] = ts_count
      else:
        break
      i += time_window_size
      ts_count += 1

    df.to_csv(f"divided600/{path_dir}/{file}", index=False)

In [ ]:
os.mkdir("divided900")

### hiperparameter
time_window_size = 90

for f in range(0, 9):
  if f == 7:
    continue
  path_dir = str(f)
  os.mkdir(f"divided900/{f}")
  for file in os.listdir(f"downsampled/{f}"):
    if file.startswith('DRAWN'):
      continue
    df = pd.read_csv(f"downsampled/{path_dir}/{file}")
    df.drop(columns=['P-JUS-CKGL', 'T-JUS-CKGL', 'QGL'], inplace=True)
    df['id'] = 0
    row_count = len(df.index)
    ts_count = 1
    i = 0
    while i < row_count:
      if i + time_window_size < row_count:
        if df['class'].iloc[i] != df['class'].iloc[i+time_window_size]:
          tmp = df.loc[i:(i+time_window_size)]
          maxi = tmp[['class']].value_counts().index[0]
          tmp.loc[df['class'] == maxi, 'id'] = ts_count
          df.loc[i:(i+time_window_size), :] = tmp[:]
        else:
          df.loc[i:(i+time_window_size), 'id'] = ts_count
      elif row_count - i >= time_window_size / 2:
        df.loc[i:(row_count-1), 'id'] = ts_count
      else:
        break
      i += time_window_size
      ts_count += 1

    df.to_csv(f"divided900/{path_dir}/{file}", index=False)

delete rows with id 0

In [7]:
l = [30, 60, 90]

for s in l:
  os.mkdir(f"final{s}0")

  for f in range(0, 9):
    if f == 7:
      continue
    path_dir = str(f)
    os.mkdir(f"final{s}0/{path_dir}")
    for file in os.listdir(f"divided{s}0/{path_dir}"):
        df = pd.read_csv(f"divided{s}0/{path_dir}/{file}")
        df = df[df.id != 0]
        df.to_csv(f"final{s}0/{path_dir}/{file}", index=False)

### Feature extraction

In [ ]:
l = [30, 60, 90]


for s in l:
  os.mkdir(f"features{s}0")

  for f in range(0, 9):
    if f == 7:
      continue
    path_dir = str(f)
    os.mkdir(f"features{s}0/{path_dir}")
    for file in os.listdir(f"final{s}0/{path_dir}"):
        df = pd.read_csv(f"final{s}0/{path_dir}/{file}")
        try:
          df = impute(df)
          new_df = extract_features(df, column_id="id", default_fc_parameters=fc_parameters, disable_progressbar=True)
          new_df.to_csv(f"features{s}0/{path_dir}/{file}", index=False)
        except ZeroDivisionError:
          print(file)

### Feature extraction finalization
** everything from now on was done only for 300 seconds time window due to lack of time to execute complex functions, but the procedure is completely same for 600 and 900 seconds time windows

In [10]:
l = [300]

for s in l:
  os.mkdir(f"final_features{s}")

  for f in range(0, 9):
    if f == 7:
      continue
    path_dir = str(f)
    os.mkdir(f"final_features{s}/{path_dir}")
    for file in os.listdir(f"features{s}/{path_dir}"):
        df = pd.read_csv(f"features{s}/{path_dir}/{file}")
        df = df.rename(columns={'class__mean': 'table_class'})
        cols = [c for c in df.columns if not c.lower().startswith('class')]
        df=df[cols]
        df['class'] = f
        df.to_csv(f"final_features{s}/{path_dir}/{file}", index=False)

# random file just to make sure it looks right
df = pd.read_csv("final_features300/1/SIMULATED_00009.csv")
df.tail()

,P-PDG__mean,P-PDG__median,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,"P-PDG__fft_aggregated__aggtype_""centroid""","P-PDG__fft_aggregated__aggtype_""variance""","P-PDG__fft_aggregated__aggtype_""skew""","P-PDG__fft_aggregated__aggtype_""kurtosis""",P-PDG__maximum,P-PDG__minimum,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-PDG__friedrich_coefficients__coeff_1__m_1__r_30,P-PDG__friedrich_coefficients__coeff_1__m_3__r_30,P-TPT__mean,P-TPT__median,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,"P-TPT__fft_aggregated__aggtype_""centroid""","P-TPT__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""skew""","P-TPT__fft_aggregated__aggtype_""kurtosis""",P-TPT__maximum,P-TPT__minimum,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,...,P-MON-CKP__minimum,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,P-MON-CKP__friedrich_coefficients__coeff_1__m_1__r_30,P-MON-CKP__friedrich_coefficients__coeff_1__m_3__r_30,T-JUS-CKP__mean,T-JUS-CKP__median,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,"T-JUS-CKP__fft_aggregated__aggtype_""centroid""","T-JUS-CKP__fft_aggregated__aggtype_""variance""","T-JUS-CKP__fft_aggregated__aggtype_""skew""","T-JUS-CKP__fft_aggregated__aggtype_""kurtosis""",T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,T-JUS-CKP__friedrich_coefficients__coeff_1__m_1__r_30,T-JUS-CKP__friedrich_coefficients__coeff_1__m_3__r_30,table_class,class
291,26905750.0,26905750.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,NaN,NaN,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,0.0,0.0,0.0,NaN,NaN,15709760.0,15709760.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,NaN,NaN,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,...,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,0.0,0.0,0.0,NaN,NaN,84.858910,84.85891,8.077936e-28,0.0,0.0,1.764215e-16,1.411911e-15,NaN,NaN,84.85891,84.858910,84.85891,84.85891,84.85891,84.85891,84.85891,84.85891,84.85891,84.85891,3.349290e-16,0.000000e+00,0.000000e+00,NaN,NaN,1.0,1
292,26905750.0,26905750.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,NaN,NaN,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,0.0,0.0,0.0,NaN,NaN,15709760.0,15709760.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,NaN,NaN,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,...,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,1151445.0,0.0,0.0,0.0,NaN,NaN,84.858917,84.85892,1.941000e-11,0.0,0.0,3.998487e-07,3.188985e-06,NaN,NaN,84.85892,84.858910,84.85891,84.85891,84.85892,84.85892,84.85892,84.85892,84.85892,84.85892,5.191768e-08,3.448276e-07,0.000000e+00,NaN,NaN,1.0,1
293,26905750.0,26905750.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,NaN,NaN,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,26905750.0,0.0,0.0,0.0,NaN,NaN,15709760.0,15709760.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,NaN,NaN,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,15709760.0,...,1151445.0,1151445.0,1151

All time windows put into one file (dataframe)

In [11]:
os.mkdir("one_csv300")

df = pd.DataFrame()
for f in range(0, 9):
  if f == 7:
    continue
  path_dir = str(f)
  for file in os.listdir(f"final_features300/{path_dir}"):
      tmp_df = pd.read_csv(f"final_features300/{path_dir}/{file}")
      df = df.append(tmp_df)
df.to_csv("one_csv300/all.csv", index=False)

print(len(df.index))

159256


Remove features with more than 10% NaN values and correct left over NaN values

In [67]:
ndf = pd.read_csv("one_csv300/all.csv")

limitPer = len(ndf.index) * .90
df = ndf.dropna(thresh=limitPer, axis=1)

df = impute(df)


y_col = df.loc[:,'class']
y = y_col.values

X = df.drop(columns=['class', 'table_class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Needed for later
n_of_features = len(X_train.columns)

### Feature selection

In [29]:
X_selected = select_features(X_train, y_train)
X_selected_train = X_selected

X_selected_test = X_test[list(X_selected_train.columns)]

X_selected_train.head()

,P-PDG__mean,P-TPT__quantile__q_0.8,P-MON-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__minimum,T-JUS-CKP__maximum,"T-JUS-CKP__fft_aggregated__aggtype_""centroid""",T-JUS-CKP__kurtosis,P-TPT__quantile__q_0.7,T-JUS-CKP__median,P-MON-CKP__variation_coefficient,P-MON-CKP__quantile__q_0.9,P-MON-CKP__mean,P-MON-CKP__median,P-MON-CKP__variance,P-MON-CKP__quantile__q_0.8,"P-MON-CKP__fft_aggregated__aggtype_""centroid""","P-MON-CKP__fft_aggregated__aggtype_""variance""",P-MON-CKP__maximum,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,T-JUS-CKP__mean,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.9,T-JUS-CKP__quantile__q_0.9,P-PDG__quantile__q_0.9,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.3,P-PDG__mean_change,P-PDG__quantile__q_0.2,P-PDG__minimum,...,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.1,T-JUS-CKP__mean_change,P-TPT__minimum,P-MON-CKP__quantile__q_0.1,T-TPT__variance,"T-JUS-CKP__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""variance""",T-TPT__minimum,T-TPT__quantile__q_0.1,"P-TPT__fft_aggregated__aggtype_""centroid""",T-TPT__quantile__q_0.2,P-TPT__variance,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__mean,T-TPT__median,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__maximum,T-TPT__kurtosis,P-MON-CKP__minimum,P-TPT__variation_coefficient,P-TPT__mean_change,T-JUS-CKP__variance,P-PDG__mean_second_derivative_central,P-PDG__skewness,P-MON-CKP__skewness,T-TPT__mean_change,P-TPT__mean_second_derivative_central,T-JUS-CKP__mean_second_derivative_central,P-TPT__skewness,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,T-TPT__mean_second_derivative_central,T-JUS-CKP__skewness,T-TPT__skewness
17406,0.000000e+00,8450720.76,1486365.00,76.595661,76.425254,76.032046,75.983854,75.965904,75.754311,75.418893,77.037954,0.038310,-1.061431,8449157.23,76.212441,0.167693,1910759.08,1.559960e+06,1510773.55,6.843130e+10,1789322.60,1.223463,8.004750,2070159.8,1723183.35,1386606.38,1441544.30,76.298379,8447593.62,8452284.52,76.946960,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,8442902.87,8441339.32,8440058.10,-0.033675,8438883.4,1186896.34,1.817536e-27,0.288286,0.038795,117.418700,117.418700,0.004713,117.418700,2.085221e+07,117.418700,117.418700,117.418700,117.418700,117.418700,117.418700,117.418700,117.418700,117.41870,0.000000,1044483.6,0.000541,-474.641379,0.210461,0.000000,0.000000,-0.140843,0.000000,31.060714,0.002811,0.060054,21021.503448,4851.892857,0.000000,0.081218,0.000000
61303,2.533910e+07,14705776.60,1150536.60,84.550734,84.547923,84.542914,84.540731,84.538770,84.537020,84.535475,84.560172,0.000856,-1.138874,14701653.10,84.545312,0.001249,1153101.82,1.151106e+06,1151014.30,2.067932e+06,1152560.08,0.012105,0.103935,1153654.0,1152029.27,1149654.60,1150082.62,84.546224,14697374.60,14709713.30,84.556885,25345148.2,25343620.8,25342109.4,25340606.2,25337602.0,25336090.1,-524.896552,25334567.4,25331479.0,...,14683989.60,14679488.20,14675056.80,0.000852,14670751.0,1149254.58,1.535651e-07,0.007356,0.072890,97.998103,97.998465,0.008491,97.998771,1.672978e+08,97.999017,97.999202,97.999100,97.999328,97.999359,97.999376,97.999395,97.999413,97.99942,0.393255,1148884.0,0.000880,1472.172414,0.000056,0.053571,-0.003655,0.156876,0.000044,-3.625000,0.000011,-0.060737,164.482759,1.200000,-0.000002,0.299220,-1.258982
71781,1.724138e+07,9988361.10,4072994.58,82.518932,82.302582,81.939266,81.762797,81.586157,81.411432,81.239859,83.343335,0.073964,-1.007254,9982512.19,82.116802,0.000782,4077904.21,4.073798e+06,4074002.35,1.014293e+07,4076943.78,0.007839,0.067556,4078831.8,4075961.32,4070678.20,4071891.91,82.177172,9973937.68,9993623.39,83.044790,17261308.1,17259036.6,17255662.2,17252306.4,17242415

### Scaling

In [ ]:
# Normalizes the samples

scaled_features = scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(scaled_features, index=X_train.index, columns=X_train.columns)

scaled_features = scaler.transform(X_test)
X_test = pd.DataFrame(scaled_features, index=X_test.index, columns=X_test.columns)

In [ ]:
# Same for feature selection data

scaled_features = scaler.fit_transform(X_selected_train.values)
X_selected_train = pd.DataFrame(scaled_features, index=X_selected_train.index, columns=X_selected_train.columns)

scaled_features = scaler.transform(X_selected_test)
X_selected_test = pd.DataFrame(scaled_features, index=X_selected_test.index, columns=X_selected_test.columns)

# Classification

In [53]:
results = pd.DataFrame(columns=['classifier', 'f1', 'accuracy', 'time_window', 'hiperparameters'])

## Logistic regression

In [55]:
cv = 5

param_grid = [{'alpha': list(math.exp(i) for i in range(-7, 0))}]

model = SGDClassifier(loss='log')
clf = GridSearchCV(model, param_grid, cv=cv, scoring='f1_macro')
clf.fit(X_train, y_train)
model = clf.best_estimator_

print(clf.best_params_)
print(clf.best_score_)


{'alpha': 0.0009118819655545162}
0.7905413838797111


In [56]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'Logistic', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': clf.best_params_['alpha']}

results = results.append(df2, ignore_index=True)



Accuracy: 0.8581116436779204
F1: 0.7977158988276813


## SVC

In [57]:
cv = 5

param_grid = [{'alpha': list(math.exp(i) for i in range(-7, 0))}]

model = SGDClassifier(loss='hinge')

clf = GridSearchCV(model, param_grid, cv=cv, scoring='f1_macro')
clf.fit(X_train, y_train)
model = clf.best_estimator_

print(clf.best_params_)
print(clf.best_score_)



{'alpha': 0.0009118819655545162}
0.7772198015120644


In [58]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'SVC', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': clf.best_params_['alpha']}

results = results.append(df2, ignore_index=True)



Accuracy: 0.8460974946103774
F1: 0.7757979608102408


## LDA and QDA

### LDA

In [59]:
model = LinearDiscriminantAnalysis()

model.fit(X_train, y_train)


LinearDiscriminantAnalysis()

In [60]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'LDA', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': ""}

results = results.append(df2, ignore_index=True)



Accuracy: 0.8329112334386839
F1: 0.7380575155615389


### QDA

In [ ]:
model = QuadraticDiscriminantAnalysis()

model.fit(X_train, y_train)


In [62]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'QDA', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': ""}

results = results.append(df2, ignore_index=True)



Accuracy: 0.6792180337819453
F1: 0.603799639457796


## Decision Tree

In [65]:
cv = 5

param_grid = [{'ccp_alpha': list(math.exp(i) for i in range(-5, 0))}]

model = DecisionTreeClassifier()

clf = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro')
clf.fit(X_train, y_train)
model = clf.best_estimator_

print(clf.best_params_)
print(clf.best_score_)

In [66]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'Decision Tree', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': clf.best_params_['ccp_alpha']}

results = results.append(df2, ignore_index=True)



Accuracy: 0.9462921489419596
F1: 0.9190033407343818


## Random Forest

In [ ]:
cv = 5

param_grid = [{'n_estimators': [50, 100, 150, 175], 'max_depth': [5, 7, 10, None], 'max_features': [5, math.sqrt(n_of_features), 15]}]

model = RandomForestClassifier()

clf = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro')
clf.fit(X_train, y_train)
model = clf.best_estimator_

print(clf.best_params_)
print(clf.best_score_)

In [ ]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'Random Forest', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': f"Max depth: {clf.best_params_['max_depth']}, max features: {clf.best_params_['max_features']}, num trees: {clf.best_params_['n_estimators']}"}

results = results.append(df2, ignore_index=True)



## AdaBoost

In [ ]:
cv = 5

param_grid = [{'n_estimators': [100, 250, 400, 550], 'max_depth': [1, 3, 5], 'learning_rate': [0.01, 0.1, 1]}]

model = AdaBoostClassifier()

clf = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro')
clf.fit(X_train, y_train)
model = clf.best_estimator_

print(clf.best_params_)
print(clf.best_score_)

In [ ]:
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='macro')
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")
print(f"F1: {f1}")

df2 = {'classifier': 'AdaBoost', 'f1': f1, 'accuracy': acc, 'time_window': 300, 'hiperparameters': f"Max depth: {clf.best_params_['max_depth']}, num trees: {clf.best_params_['n_estimators']}, learning rate: {clf.best_params_['learning_rate']}"}

results = results.append(df2, ignore_index=True)

Vrsta istraživanja provedena u članku prema:
  - svrsi: Primijenjeno istraživanje - istraživanje nepoželjnih događaja u radu naftnih bušotina u svrhu izrade modela koji može predvidjeti koji nepoželjan događaj se događa u danom trenutku na osnovu podataka očitanih sa senzora
  - izvoru informacija: Sekundarno istraživanje - korišten je ranije sastavljen skup podataka od strane drugih/e osoba/e
  - vremenu provedbe: Longitudinalno istraživanje - promatraju se vrijednosti dobivene od senzora u određenom vremenskom razdoblju
  - načinu prikupljanja: Dokumentarno istraživanje - sustavno su pregledane postojeće informacije i postojeći podaci o temi
  - vrsti korištenih podataka: Kvantitativno istraživanje - prikupljeni podaci su direktno očitani sa senzora pa tako skup podataka sadrži isključivo numeričke vrijednosti
  - stupnju manipulacije varijablama: Promatračko istraživanje - u članku su samo korišteni podaci koji su već postojali, ako gledamo same podatke, mogli bi reći da ima i elemenata kvazi-eksperimentalnog istraživanja jer su dodani mnogi simulirani podaci
  - dubini: elementi Objašnjavačkog i Korelacijskog istraživanja - cilj je definirati odnos između senzorskih varijabli i nepoželjnih događaja, ali i uspostavljanje uzročno-posljedičnih veza i pružanje dodatnih informacija o temi koju članak obrađuje
  - vrsti zaključka: Deduktivno istraživanje - samom izradom modela za klasifikaciju uvode se poveznice i zaključci o odnosu između ulaznih varijabli i događaja koji se predviđa

Način na koji su podaci za članak prikupljeni najviše nalikuje operacijskom istraživanju jer se promatraju događaji koji se stvarno događaju u radu naftnih bušotina.

Podaci su prikupljeni na etičan način. Nisam pronašao da je navedeno sa kojih je naftnih bušotina obavljeno prikupljanje podataka. To je dobro jer bi se u suprotnom možda moglo iskoristiti znanja iz članka za namjerno prouzročiti probleme da danoj naftnoj bušotini.
